# Cubing steps with PyKOALA

This tutorial is currently used for testing purposes.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from astropy import units as u
import os
from pykoala import __version__
import warnings

# You may want to comment the following line
warnings.filterwarnings("ignore")

from pykoala.instruments.koala_ifu import koala_rss
from pykoala.cubing import CubeInterpolator, build_wcs
# pyKOALA version
print("pyKOALA version: ", __version__)

## Load the data

In [ ]:
# List of RSS objects
std_star_rss = []
aaomega_arms = {'blue': 1, 'red': 2}
# Choose which arm of the spectrograph is going to be used
arm = 'red'
path_to_data = 'data'

for i in [28, 29, 30]:
    filename = f"{path_to_data}/27feb{aaomega_arms[arm]}00{i}red.fits"
    rss = koala_rss(filename)
    std_star_rss.append(rss)

star_name = rss.info['name'].split(' ')[0]
print("Star name: ", star_name)

In [ ]:
from pykoala.corrections.astrometry import AstrometryCorrection
from pykoala.corrections.astrometry import register_dataset_centroids, register_dataset_crosscorr

astrom_corr = AstrometryCorrection()

offsets, fig = register_dataset_centroids(std_star_rss, object_name=star_name,
                                          qc_plot=True, centroider='gauss')

In [ ]:
fig

In [ ]:
# for rss, offset in zip(std_star_rss, offsets):
#     astrom_corr.apply(rss, offset=offset)

# # Check that the corrections has been logged
# print(rss.history)

In [ ]:
offsets, fig = register_dataset_crosscorr(std_star_rss, qc_plot=True)

In [ ]:
fig

For interpolating RSS data into a 3D datacube we will make use of the function *build_cube*. This method requires as input:
- A list of RSS objects. 
- The desired dimensions of the cube expressed as a 2-element tuple, corresponding to (ra, dec) in arcseconds.
- The pixel size of the cube in arcseconds.
- A list containing the ADR correction for every RSS (it can contain None) in the form: [(ADR_ra_1, ADR_dec_1), (ADR_ra_2, ADR_dec_2), (None, None)].
- Additional information to be included in *cube_info*

In [ ]:
datacube_shape = (std_star_rss[0].wavelength.size, 40, 60)
ref_position = (std_star_rss[0].wavelength[0], np.mean(std_star_rss[0].info['fib_ra']), np.mean(std_star_rss[0].info['fib_dec']))  # (deg, deg)
spatial_pixel_size = 1.0 << u.arcsec
spectral_pixel_size = std_star_rss[0].wavelength[1] - std_star_rss[0].wavelength[0]  # (angstrom)

print(f"Creating a WCS with\n position: {ref_position}\n Spatial pixel size: {spatial_pixel_size}\n Spectral pixel size: {spectral_pixel_size}")

wcs = build_wcs(datacube_shape=datacube_shape,
                reference_position=ref_position,
                spatial_pix_size=spatial_pixel_size,
                spectra_pix_size=spectral_pixel_size,
            )

In [ ]:
interpolator = CubeInterpolator(rss_set=std_star_rss, wcs=wcs, kernel_scale=1.0)
cube = interpolator.build_cube()
white_image = np.nanmean(cube.intensity, axis=0)

In [ ]:
from pykoala.corrections.astrometry import find_centroid_in_dc

pos_com = find_centroid_in_dc(cube, centroider='com', com_power=1.)
pos_com_3 = find_centroid_in_dc(cube, centroider='com', com_power=3.)
pos_gauss = find_centroid_in_dc(cube, centroider='gauss')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection=cube.wcs.celestial)
ax.imshow(np.log10(white_image.value))
ax.scatter(pos_com.ra, pos_com.dec, marker='*', ec='r', transform=ax.get_transform('world'))
ax.scatter(pos_com_3.ra, pos_com_3.dec, marker='*', ec='lime', transform=ax.get_transform('world'))
ax.scatter(pos_gauss.ra, pos_gauss.dec, marker='+', ec='k', transform=ax.get_transform('world'))
